In [1]:
from src.code_gen.code_generator import CCodeGenerator

generator = CCodeGenerator()

In [2]:
code = generator('''
    protocol ABC {
        getsum(s: Number): String;
    }

    type C (cc : Number) {
        x = cc;
                          
        getsum(s: Number) : String => let a = 5 in s + a;
    }

    type B inherits C {
        f : Number = cc * cc / cc;
        g : String = "qwerty";
    }

    type A(q : Number, r : Number) inherits B (q * r + 5) {
        f : Number = q;
        p : Number = r;
    }

    function operate(x : Number, y : Object) : String {
        print((x + (y as Number)) as Object);
    }
                       
    function lambda(a : A) {
        print(a.getsum(5));
    }
                       
    let a : Number = 6 in
    let b : Number = a * 7, c : Object = b in
        lambda(new A(a, b));
    '''
)

print(code)

Object* createC (Object* C_x);

Object* createB (Object* B_f, Object* B_g, Object* C_x);
Object* method_B_getsum (Object* self, Object* p0);

Object* createA (Object* A_f, Object* A_p, Object* B_f, Object* B_g, Object* C_x);

Object* function_operate (Object* p0, Object* p1);
Object* function_lambda (Object* p0);

Object* createC (Object* C_x) {
   Object* obj = createObject();

   addAttribute(obj, "C_x", C_x);

   addAttribute(obj, "parent_type0","C");
   addAttribute(obj, "parent_type1","Object");
   return obj;
}

Object* createB (Object* B_f, Object* B_g, Object* C_x) {
   Object* obj = createObject();

   addAttribute(obj, "B_f", B_f);
   addAttribute(obj, "B_g", B_g);
   addAttribute(obj, "C_x", C_x);

   addAttribute(obj, "parent_type0","B");
   addAttribute(obj, "method_B_getsum", *method_B_getsum);
   addAttribute(obj, "parent_type1","C");
   addAttribute(obj, "parent_type2","Object");
   return obj;
}

Object* createA (Object* A_f, Object* A_p, Object* B_f, Object* B_g, Obje